In [1]:
import pandas as pd
import math
import numpy as np

In [2]:
import pickle
XX = pickle.load(open("bias_P&V_15\2\3\5\20_X","rb"))
YY = pickle.load(open("bias_P&V_15\2\3\5\20_Y","rb"))

In [3]:
from sklearn.utils import shuffle
XX, YY = shuffle(XX, YY, random_state=1)

train_x = XX[:10000,:,:,1]
test_x = XX[10000:,:,:,1]
train_y = YY[:10000,:]
test_y = YY[10000:,:]

print(train_x.shape)
print(test_y.shape)


(10000, 20, 20)
(828, 3)


In [4]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [5]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID') #1x1 patch stride = 1

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], #Due to 2x2, half output size
                        strides=[1, 2, 2, 1], padding='VALID')
        

In [6]:
x = tf.placeholder(tf.float32, shape=[None, 20,20])
y_ = tf.placeholder(tf.float32, shape=[None, 3])

In [7]:
W_conv1 = weight_variable([5, 5, 1, 32]) #index 2: input size(depth = 5); index 3:output size(depth = 32)
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,20,20,1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

print(h_conv1)
print(h_pool1)


Tensor("Relu:0", shape=(?, 16, 16, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 8, 8, 32), dtype=float32)


In [8]:
W_conv2 = weight_variable([3, 3, 32, 128])
b_conv2 = bias_variable([128])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

print(h_conv2)
print(h_pool2)

Tensor("Relu_1:0", shape=(?, 6, 6, 128), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 3, 3, 128), dtype=float32)


In [9]:
W_fc1 = weight_variable([3 * 3 * 128, 512])
b_fc1 = bias_variable([512])

h_pool2_flat = tf.reshape(h_pool2, [-1, 3 * 3 * 128])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([512, 3])
b_fc2 = bias_variable([3])
y_conv= tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


In [10]:
print(x)
print(x_image)
print(h_conv1)
print(h_pool1)
print(h_conv2)
print(h_pool2)
print(h_pool2_flat)
print(h_fc1)
print(h_fc1_drop)
print(y_conv)

Tensor("Placeholder:0", shape=(?, 20, 20), dtype=float32)
Tensor("Reshape:0", shape=(?, 20, 20, 1), dtype=float32)
Tensor("Relu:0", shape=(?, 16, 16, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 8, 8, 32), dtype=float32)
Tensor("Relu_1:0", shape=(?, 6, 6, 128), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 3, 3, 128), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 1152), dtype=float32)
Tensor("Relu_2:0", shape=(?, 512), dtype=float32)
Tensor("dropout/mul:0", shape=(?, 512), dtype=float32)
Tensor("Softmax:0", shape=(?, 3), dtype=float32)


In [11]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) 

test= tf.log(y_conv)

In [12]:
sess.run(tf.initialize_all_variables())
for i in range(10000):
    train_x, train_y = shuffle(train_x, train_y, random_state=2)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:train_x, y_: train_y, keep_prob: 0.8})

        print("step %d, training accuracy %g"%(i, train_accuracy))
        print("test accuracy %g"%accuracy.eval(feed_dict={
        x: test_x, y_: test_y, keep_prob: 0.8}))
        
        print("cross_entropy %g"%cross_entropy.eval(feed_dict={
        x: test_x, y_: test_y, keep_prob: 0.8}))
        
        print(test.eval(feed_dict={
        x: train_x, y_: train_y, keep_prob: 0.8}))
            

    train_step.run(feed_dict={x: train_x[:200], y_: train_y[:200], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: test_x, y_: test_y, keep_prob: 0.8}))

step 0, training accuracy 0.318
test accuracy 0.326087
cross_entropy 1.45801
[[-0.19624119 -2.05076861 -3.00477266]
 [-0.38928115 -3.43721485 -1.23683286]
 [-1.23439324 -0.93247616 -1.15387952]
 ..., 
 [-0.16373186 -2.82638884 -2.38809919]
 [-0.62571502 -1.55236685 -1.37288511]
 [-0.49570405 -1.0023638  -3.73608732]]
step 100, training accuracy 0.4556
test accuracy 0.434783
cross_entropy 1.07568
[[-1.26152325 -1.57968366 -0.67189956]
 [-1.24116468 -1.54344344 -0.6985448 ]
 [-1.248546   -1.4645189  -0.73004127]
 ..., 
 [-1.20422626 -1.51953506 -0.73134261]
 [-1.22993016 -1.55403137 -0.70058906]
 [-1.24942815 -1.47287178 -0.72553581]]
step 200, training accuracy 0.4556
test accuracy 0.434783
cross_entropy 1.06972
[[-1.12245297 -1.46980512 -0.81069434]
 [-1.12515056 -1.41333759 -0.83917546]
 [-1.11747611 -1.47292936 -0.8127358 ]
 ..., 
 [-1.16599524 -1.32473576 -0.86153173]
 [-1.19760442 -1.45451605 -0.76664287]
 [-1.12214804 -1.37965882 -0.86096025]]
step 300, training accuracy 0.4556
te

In [13]:
sess.close()

In [14]:
A= sum(train_y[:,0] ==1)
B =sum(train_y[:,1] ==1)
C= sum(train_y[:,2] ==1)
print(A)
print(B)
print(C)

3150
2294
4556


In [15]:
A= sum(test_y[:,0] ==1)
B =sum(test_y[:,1] ==1)
C= sum(test_y[:,2] ==1)
print(A)
print(B)
print(C)

271
197
360
